In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

I am building a tool that reads raw emails and decides whether that email is spam or ham. The main goal is for the classifier to be reliable so that users can focus on real emails instead of spam cluttering their inbox.

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

The data was provided to me on Schoology as a zip file. The unpacked zip file shows a folder of text files. Each file is one per email, and the first line usually holds the subject, while the rest is the body. 

In [16]:
from pathlib import Path
import pandas as pd
import re

data_dir = Path("email_archive")

def read_email(path: Path):
    text = path.read_text(errors="ignore")
    lines = text.splitlines()
    subject = ""
    body_lines = []
    for i, line in enumerate(lines):
        if i == 0 and re.match(r'^\s*subject', line, re.I):
            subject = re.sub(r'^\s*subject\s*', '', line, flags=re.I).strip()
        else:
            body_lines.append(line)
    body = "\n".join(body_lines).strip()
    return subject, body

rows = []
for p in sorted(data_dir.glob("**/*.txt")):
    subj, body = read_email(p)
    rows.append({"path": str(p), "subject": subj, "body": body})

df = pd.DataFrame(rows)
print(df.shape)
df.head()


(0, 0)


""


# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

At first, I analyzed the basic aspects of the data. This includes how many emails there are, how long the subjects are as well as the bodies. I also checked if there is any heavy quoting or if tehre are many addresses or links in the body of the emails. 

In [18]:
from pathlib import Path
import pandas as pd
import re

data_dir = Path("email_archive")

def read_email(path: Path):
    text = path.read_text(encoding="utf-8", errors="ignore")
    lines = text.splitlines()

    # find subject anywhere in the first 10 lines
    subj_idx = None
    subject = ""
    for i, line in enumerate(lines[:10]):
        m = re.match(r'^\s*subject\s*(.*)$', line, flags=re.I)
        if m:
            subject = m.group(1).strip()
            subj_idx = i
            break

    # build body without the subject line if found
    if subj_idx is not None:
        body_lines = lines[:subj_idx] + lines[subj_idx+1:]
    else:
        body_lines = lines
    body = "\n".join(body_lines).strip()

    return subject, body

rows = []
txt_files = sorted(data_dir.glob("**/*.txt"))
for p in txt_files:
    subj, body = read_email(p)
    rows.append({"path": str(p), "subject": subj, "body": body})

df = pd.DataFrame(rows)
print("Loaded files", len(txt_files), "parsed rows", len(df))
display(df.head())


Loaded files 0 parsed rows 0


""


# 4.Prepare the Data


Apply any data transformations and explain what and why


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


Chosen A: LinearRegression
Chosen B: LinearRegression


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


Saved: artifacts/model_A_sex.joblib artifacts/model_B_pclass.joblib artifacts/metrics.json
Accuracy A (sex) @0.5: 0.788
Accuracy B (pclass) @0.5: 0.676


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 
